In [ ]:
import pyemu
import os
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import matplotlib.pyplot as plt
import psutil
import shutil
import numpy as np
import sys

In [ ]:
path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.insert(1, path)
from swatmf import swatmf_pst_utils, swatmf_viz

# 01. Set working directory

In [ ]:
# working directory
wd = "D:/spark/mb/main_zon_rw_ies"
os.chdir(wd)

# 02. Initial Run

In [ ]:
# test run
pyemu.os_utils.run("pestpp-ies mb_zon.pst",cwd=wd)

# 03. Reweight

In [ ]:
# reweight
pst = pyemu.Pst(os.path.join(wd,'mb_zon.pst'))
pst.phi


In [ ]:
balanced_groups = {grp:10000 for grp in pst.nnz_obs_groups}
# print(pst.phi) #assumes "my.res" is colocated with "my.pst"
pst.adjust_weights(obsgrp_dict=balanced_groups)

In [ ]:
os.getcwd()

In [ ]:
obs = pst.observation_data.copy()

In [ ]:
pst.observation_data = obs

In [ ]:
pst.write('mb_zon_rw.pst', version=2)

# 04. Run IES

## 04.1 Set up IES

In [ ]:
# check number of cores on your computer
num_workers = psutil.cpu_count(logical=False)

In [ ]:
pst_ies = pyemu.Pst(os.path.join(wd,"mb_zon_rw.pst"))
# set IES
pst_ies.pestpp_options['ies_num_reals'] = 300 # number of realization
pst_ies.control_data.noptmax = 5 # number of iteration
pst_ies.write('mb_zon_rw_ies.pst', version=2) # write new IES control file

In [ ]:
os.chdir(os.pardir)

In [ ]:
os.getcwd()

In [ ]:
m_d = os.path.join(os.getcwd(), "main_zon_rw_ies02")

## 04.2 Execute

In [ ]:
pyemu.os_utils.start_workers(wd, # the folder which contains the "template" PEST dataset
                            'pestpp-ies', #the PEST software version we want to run
                            'mb_zon_rw_ies.pst', # the control file to use with PEST
                            num_workers=num_workers, #how many agents to deploy
                            worker_root='.', #where to deploy the agent directories; relative to where python is running
                            master_dir=m_d, #the manager directory,
                            # reuse_master=True
                            )

# 05. Analyze results
## 05.1 Check model performance

In [ ]:
pst = pyemu.Pst(os.path.join(m_d,'mb_zon_rw_ies.pst')) # load control file

In [ ]:
# load prior simulation
pr_oe = pyemu.ObservationEnsemble.from_csv(
    pst=pst,filename=os.path.join(m_d,"mb_zon_rw_ies.0.obs.csv")
    )
# load posterior simulation
pt_oe = pyemu.ObservationEnsemble.from_csv(pst=pst,filename=os.path.join(m_d,"mb_zon_rw_ies.{0}.obs.csv".format(pst.control_data.noptmax)))


In [ ]:
# plot 1 to 1 scatter plot and residuals
pyemu.plot_utils.res_1to1(pst);

In [ ]:
# check phi values
pt_oe.phi_vector

In [ ]:
# plot progress
fig,ax = plt.subplots(1,1)
pr_oe.phi_vector.apply(np.log10).hist(ax=ax,fc="0.5",ec="none",alpha=0.5,density=False)
pt_oe.phi_vector.apply(np.log10).hist(ax=ax,fc="b",ec="none",alpha=0.5,density=False)
_ = ax.set_xlabel("$log_{10}\\phi$")

## 05.2 Prior and Posterior parameter probability

In [ ]:
prior_df = pyemu.ParameterEnsemble.from_csv(pst=pst,filename=os.path.join(m_d,"mb_zon_rw_ies.{0}.par.csv".format(0)))
post_df = pyemu.ParameterEnsemble.from_csv(pst=pst,filename=os.path.join(m_d,"mb_zon_rw_ies.{0}.par.csv".format(pst.control_data.noptmax)))

In [ ]:
df_pars = pd.read_csv(os.path.join(m_d, "mb_zon_rw_ies.par_data.csv"))
sel_pars = df_pars.loc[df_pars["partrans"]=='log']
sel_pars

In [ ]:
swatmf_viz.plot_prior_posterior_par_hist(prior_df, post_df, sel_pars)

## 05.3 Predictive Uncertainty

In [ ]:
swatmf_viz.plot_tseries_ensembles(pst, pr_oe, pt_oe, height=6, dot=True)

In [ ]:
os.getcwd()

In [ ]:
pst.parrep(parfile=os.path.join(m_d, "mb_zon_rw_ies.{0}.base.csv".format(pst.control_data.noptmax)))

In [ ]:
# updates the model input files with parameter values
pst.write_input_files(pst_path=m_d)

In [ ]:
# run the model forward run; this applies all the SWAT and MODFLOW paarameters, executes SWAT-MODFLOW 
os.chdir(m_d)
pyemu.os_utils.run('python forward_run.py')